# <center> <img src="../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Carrera: <Coloca el nombre de tu carrera aqui>** </center>
---
### <center> **Primavera 2025** </center>
---

**Lab XX**:

**Fecha**:

**Nombre del Estudiante**:

**Profesor**: Pablo Camarillo Ramirez

In [29]:
import findspark
findspark.init()

#### Creacion de la conexión con el cluster de spark


In [30]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQLStructuredStreaming-Kafka-Watermarking") \
    .master("spark://22b4d5547f27:7077") \
    .config("spark.ui.port","4040") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:3.5.4") \
    .getOrCreate()
sc = spark.sparkContext

### Creación del Kafka Stream

In [31]:
kafka_lines = spark \
                .readStream \
                .format("kafka") \
                .option("kafka.bootstrap.servers", "87e989446b6a:9093") \
                .option("subscribe", "kafka-spark-example") \
                .load()

kafka_lines.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



### Transform binary data into string

In [32]:
from pyspark.sql.functions import col, split

kafka_df = kafka_lines.withColumn("value_str", kafka_lines.value.cast("string"))

parsed_df = kafka_df.withColumn("word", split(col("value_str"), ",").getItem(0)) \
                    .withColumn("num", split(col("value_str"), ",").getItem(1).cast("int")) \
                    .withColumn("timestamp", col("timestamp"))

### Aplicando el mecanismo para manejar datos tardios con marcas de agua (watermarking)

In [35]:

from pyspark.sql.functions import window, count, sum, max
windowed_counts = parsed_df \
                    .withWatermark("timestamp", "3 minutes") \
                    .groupBy(
                        window(col("timestamp"), "90 seconds", "45 seconds"),
                        col("word")
                    ) \
                    .agg(count("*"), sum("num"), max("num"))

### Configuración del "Sink" del stream

In [36]:
spark.conf.set("spark.sql.shuffle.partitions", "5")

query = windowed_counts \
                .writeStream \
                .outputMode("update") \
                .trigger(processingTime='30 seconds') \
                .format("console") \
                .option("truncate", "false") \
                .start()

query.awaitTermination(300)

25/04/08 14:46:20 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-de8bde9e-1544-4b83-9ca6-057b10441483. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/04/08 14:46:20 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/04/08 14:46:20 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


-------------------------------------------
Batch: 0
-------------------------------------------
+------+----+--------+--------+--------+
|window|word|count(1)|sum(num)|max(num)|
+------+----+--------+--------+--------+
+------+----+--------+--------+--------+



-------------------------------------------
Batch: 1
-------------------------------------------
+------------------------------------------+-----+--------+--------+--------+
|window                                    |word |count(1)|sum(num)|max(num)|
+------------------------------------------+-----+--------+--------+--------+
|{2025-04-08 14:45:45, 2025-04-08 14:47:15}|pedro|1       |1       |1       |
|{2025-04-08 14:46:30, 2025-04-08 14:48:00}|hola |1       |2       |2       |
|{2025-04-08 14:45:45, 2025-04-08 14:47:15}|hola |1       |2       |2       |
|{2025-04-08 14:46:30, 2025-04-08 14:48:00}|pedro|1       |1       |1       |
+------------------------------------------+-----+--------+--------+--------+



-------------------------------------------
Batch: 4
-------------------------------------------
+------------------------------------------+-----+-----+-------+-------+
|window                                    |word |count|sum_num|max_num|
+------------------------------------------+-----+-----+-------+-------+
|{2025-04-08 14:46:30, 2025-04-08 14:47:30}|hola |1    |2      |2      |
|{2025-04-08 14:46:30, 2025-04-08 14:47:30}|pedro|1    |1      |1      |
|{2025-04-08 14:46:00, 2025-04-08 14:47:00}|pedro|1    |1      |1      |
|{2025-04-08 14:46:00, 2025-04-08 14:47:00}|hola |1    |2      |2      |
+------------------------------------------+-----+-----+-------+-------+



-------------------------------------------
Batch: 2
-------------------------------------------
+------------------------------------------+-----+--------+--------+--------+
|window                                    |word |count(1)|sum(num)|max(num)|
+------------------------------------------+-----+--------+--------+--------+
|{2025-04-08 14:45:45, 2025-04-08 14:47:15}|pedro|2       |20      |19      |
|{2025-04-08 14:46:30, 2025-04-08 14:48:00}|hola |3       |416     |399     |
|{2025-04-08 14:45:45, 2025-04-08 14:47:15}|hola |3       |416     |399     |
|{2025-04-08 14:46:30, 2025-04-08 14:48:00}|pedro|2       |20      |19      |
+------------------------------------------+-----+--------+--------+--------+



-------------------------------------------
Batch: 5
-------------------------------------------
+------------------------------------------+-----+-----+-------+-------+
|window                                    |word |count|sum_num|max_num|
+------------------------------------------+-----+-----+-------+-------+
|{2025-04-08 14:47:00, 2025-04-08 14:48:00}|pedro|1    |19     |19     |
|{2025-04-08 14:46:30, 2025-04-08 14:47:30}|pedro|2    |20     |19     |
|{2025-04-08 14:46:30, 2025-04-08 14:47:30}|hola |3    |416    |399    |
|{2025-04-08 14:47:00, 2025-04-08 14:48:00}|hola |2    |414    |399    |
+------------------------------------------+-----+-----+-------+-------+



-------------------------------------------
Batch: 6
-------------------------------------------
+------+----+-----+-------+-------+
|window|word|count|sum_num|max_num|
+------+----+-----+-------+-------+
+------+----+-----+-------+-------+



-------------------------------------------
Batch: 3
-------------------------------------------
+------+----+--------+--------+--------+
|window|word|count(1)|sum(num)|max(num)|
+------+----+--------+--------+--------+
+------+----+--------+--------+--------+



False

In [28]:
query.stop()
sc.stop()